In [1]:
import matplotlib.pyplot as plt 
import pandas as pd 
import seaborn as sns 
import numpy as np
import yaml
import pickle
from itertools import product, cycle
import os, sys
from tqdm.notebook import tqdm
from scipy import stats
from scipy.stats import friedmanchisquare
import scikit_posthocs as s

from pathlib import Path
sys.path.insert(0, str(Path().resolve().parents[1]))

*Analysis to perform*

1. Summary table of mean +- std of performances Dataset/Method/Metric
2. Analysis binned by total missing ratio;
3. Analysis binned by MNAR percentage; 

In [2]:
# load simulation results

simulation_path = "../../test_output/simulation_results11_01_2026.pkl"

with open(simulation_path, 'rb') as input_file:
    simulation_results = pickle.load(input_file)

In [3]:
# Load simulation configuration

with open("../../test_data/simulation_config.yaml", "r") as f:
    cfg = yaml.safe_load(f)

# conf is a tuple (dataset_id,props,mf_proportions,mnar_proportions, seed)

configs = list(product(
    cfg['dataset_ids'],
    cfg['md_param_grid']['props'],
    cfg['md_param_grid']['mf_proportions'],
    cfg['md_param_grid']['mnar_proportions'],
    range(cfg['n_runs'])
    ))

In [4]:
TESTED_METHODS = [
        'mige_no_proj',
        'mige_proj',
        #'mige_no_proj_mutual',
        #'mige_proj_mutual',
        'mica',
        'kpod',
        'sc_knn',
        'sc_mi',
        'km_knn',
        'km_si'
    ]

In [5]:
# configs of dataset = dataset_id
# dataset_id = configs[0][0]
# key_subset = {k: v for k, v in d.items() if k[0] == X} this is for dictionaries
# conf_subset = [conf for conf in configs if conf[0] == dataset_id]

cfg['dataset_ids']

[33, 45, 17, 15, 174, 544]

In [6]:
def read_pickle(path):
    with open(path, 'rb') as input_file:
        return pickle.load(input_file)

## Missing ratios binning

In [7]:
# simulation results is a dictionary structured as such:
# simulation_results[conf][metric_type] metric_type in {external_metrics, internal_metrics}
# conf is a tuple (dataset_id,props,mf_proportions,mnar_proportions, seed)
# subset dictionary for first element subset = {k: v for k, v in d.items() if k[0] == X}

In [8]:
# tested external metrics 
external_metrics = list(simulation_results[configs[0]]['external_metrics']['mige_no_proj'])
# tested internal metrics 
internal_metrics = list(simulation_results[configs[0]]['internal_metrics']['mige_no_proj'])
# tested methods
methods = list(simulation_results[configs[0]]['internal_metrics'])

In [9]:
def safe_get(v, key1, key2):
    """
    This function is a safe get with double key because some are not list but single np.nan
    """
    try:
        return v[key1][key2]
    except:
        return np.nan

def spaghetti_dataframe_(df, metric_name):
    """
    This function is specific to pivot a dataframe into long format
    """
    df_long = (
        df.T
        .assign(metric=metric_name)
        .set_index("metric", append=True)
        .stack()
    )
    return df_long

In [10]:
def combine_dataset(results_df, metrics):
    """
    From aggregated result datasets with the form: 
    {metric: {dataset: {method: <value>}}}
    """
    long_dfs = {
    metric: spaghetti_dataframe_(
            df = pd.DataFrame.from_dict(results_df[metric]),
            metric_name = metric
        )
    for metric in metrics
    }
    combined = pd.DataFrame(pd.concat([df for df in long_dfs.values()]))
    combined.index = combined.index.set_names(['dataset','metric','method'])
    combined = combined.rename({0: "value"}, axis = 'columns') 
    combined = combined.unstack(level = -1)
    return combined

# External metrics aggregates

In [11]:
# build dictionaries to store:
# - results (lists for each of the 10 runs)
# - means and stds (single value)
# dictionary form {metric: {dataset: {method: <value>}}}

external_metrics_results = {met:{d:{m:[] for m in methods} for d in cfg['dataset_ids']} for met in external_metrics}
external_metrics_means = {met:{d:{m:np.nan for m in methods} for d in cfg['dataset_ids']} for met in external_metrics}
external_metrics_stds = {met:{d:{m:np.nan for m in methods} for d in cfg['dataset_ids']} for met in external_metrics}

for k1,v1 in simulation_results.items():
    dataset_id = k1[0]
    # collect results
    for metric in external_metrics:
        for method in methods:
            #res = v1['external_metrics'][method][metric]
            res = safe_get(v1['external_metrics'],method,metric)
            external_metrics_results[metric][dataset_id][method].append(res)

# calculate means and standard deviations
for metric,v1 in external_metrics_results.items():
    for dataset_id, v2 in v1.items():
        for method,v3 in v2.items():
            external_metrics_means[metric][dataset_id][method] = np.mean(v3)
            external_metrics_stds[metric][dataset_id][method] = np.std(v3)

In [12]:
combined_external_means = combine_dataset(external_metrics_means,external_metrics)
combined_external_stds = combine_dataset(external_metrics_stds,external_metrics)
combined_external_means.tail(5)

value                                                 \
method         mige_no_proj mige_proj mige_no_proj_mutual mige_proj_mutual   
dataset metric                                                               
174     vm         0.222745  0.226622            0.205224         0.193623   
544     ami        0.279608  0.277943            0.012440         0.009432   
        ari        0.147121  0.144305            0.004416         0.003030   
        cs         0.306870  0.304684            0.120806         0.090708   
        vm         0.283061  0.281400            0.020001         0.016513   

                                                                            \
method              mica      kpod mcnm mghm    sc_knn     sc_mi    km_knn   
dataset metric                                                               
174     vm      0.068029  0.007024  NaN  NaN  0.110341  0.024869  0.070421   
544     ami     0.380185  0.313930  NaN  NaN  0.239012  0.183183  0.374867   
        ari     0.242474  0.169336  NaN  NaN  0.195698  0.136090  0.241047   
        cs      0.392084  0.347470  NaN  NaN  0.248208  0.189054  0.384430   
        vm      0.382997  0.317309  NaN  NaN  0.242451  0.186822  0.377678   

                                                  
method             km_si cca_spectral cca_kmeans  
dataset metric                                    
174     vm      0.070549     0.017827   0.061709  
544     ami     0.380528     0.265284   0.474400  
        ari     0.238648     0.206407   0.310081  
        cs      0.391843     0.278103   0.487260  
        vm      0.383332     0.268643   0.476776

## Ranks for external metrics

In [13]:
combined_external_means['value'][TESTED_METHODS].rank(axis = 'columns', ascending = False)

method          mige_no_proj  mige_proj  mica  kpod  sc_knn  sc_mi  km_knn  \
dataset metric                                                               
15      ami              2.0        1.0   4.0   8.0     7.0    6.0     5.0   
        ari              2.0        1.0   4.0   8.0     7.0    6.0     5.0   
        cs               2.0        1.0   4.0   8.0     7.0    6.0     5.0   
        vm               2.0        1.0   4.0   8.0     7.0    6.0     5.0   
17      ami              2.0        1.0   6.0   8.0     4.0    3.0     7.0   
        ari              1.0        2.0   7.0   8.0     4.0    3.0     6.0   
        cs               2.0        1.0   6.0   8.0     4.0    3.0     7.0   
        vm               2.0        1.0   6.0   8.0     4.0    3.0     7.0   
33      ami              2.0        1.0   7.0   8.0     4.0    3.0     6.0   
        ari              2.0        1.0   6.0   8.0     4.0    3.0     7.0   
        cs               2.0        1.0   7.0   8.0     4.0    3.0     6.0   
        vm               2.0        1.0   7.0   8.0     4.0    3.0     6.0   
45      ami              1.0        2.0   7.0   8.0     3.0    4.0     6.0   
        ari              1.0        2.0   6.0   8.0     3.0    4.0     7.0   
        cs               2.0        1.0   6.0   8.0     3.0    4.0     7.0   
        vm               1.0        2.0   6.0   8.0     3.0    4.0     7.0   
174     ami              2.0        1.0   6.0   8.0     3.0    7.0     5.0   
        ari              1.0        2.0   5.0   6.0     7.0    8.0     3.0   
        cs               2.0        1.0   6.0   8.0     3.0    7.0     4.0   
        vm               2.0        1.0   6.0   8.0     3.0    7.0     5.0   
544     ami              5.0        6.0   2.0   4.0     7.0    8.0     3.0   
        ari              6.0        7.0   1.0   5.0     4.0    8.0     2.0   
        cs               5.0        6.0   1.0   4.0     7.0    8.0     3.0   
        vm               5.0        6.0   2.0   4.0     7.0    8.0     3.0   

method          km_si  
dataset metric         
15      ami       3.0  
        ari       3.0  
        cs        3.0  
        vm        3.0  
17      ami       5.0  
        ari       5.0  
        cs        5.0  
        vm        5.0  
33      ami       5.0  
        ari       5.0  
        cs        5.0  
        vm        5.0  
45      ami       5.0  
        ari       5.0  
        cs        5.0  
        vm        5.0  
174     ami       4.0  
        ari       4.0  
        cs        5.0  
        vm        4.0  
544     ami       1.0  
        ari       3.0  
        cs        2.0  
        vm        1.0

In [14]:
# Calculate average rank for each method
combined_external_means['value'][TESTED_METHODS].rank(axis = 'columns', ascending = False).mean(axis=0)

method
mige_no_proj    2.333333
mige_proj       2.083333
mica            5.083333
kpod            7.291667
sc_knn          4.708333
sc_mi           5.208333
km_knn          5.291667
km_si           4.000000
dtype: float64

# Internal metrics aggregates

In [15]:
internal_metrics_results = {met:{d:{m:[] for m in methods} for d in cfg['dataset_ids']} for met in internal_metrics}
internal_metrics_means = {met:{d:{m:np.nan for m in methods} for d in cfg['dataset_ids']} for met in internal_metrics}
internal_metrics_stds = {met:{d:{m:np.nan for m in methods} for d in cfg['dataset_ids']} for met in internal_metrics}

for k1,v1 in simulation_results.items():
    dataset_id = k1[0]
    # collect results
    for metric in internal_metrics:
        for method in methods:
            #res = v1['internal_metrics'][method][metric]
            res = safe_get(v1['internal_metrics'],method,metric)
            internal_metrics_results[metric][dataset_id][method].append(res)

# calculate means and standard deviations

for metric,v1 in internal_metrics_results.items():
    for dataset_id, v2 in v1.items():
        for method,v3 in v2.items():
            internal_metrics_means[metric][dataset_id][method] = np.mean(v3)
            internal_metrics_stds[metric][dataset_id][method] = np.std(v3)

In [16]:
combined_internal_means = combine_dataset(internal_metrics_means, internal_metrics) 
combined_internal_stds = combine_dataset(internal_metrics_stds, internal_metrics)
combined_internal_means.head(5)

value                                                   \
method         mige_no_proj   mige_proj mige_no_proj_mutual mige_proj_mutual   
dataset metric                                                                 
15      ch       967.119891  966.652509            1.446482         1.624855   
        db         0.774910    0.774950            3.074198         3.887303   
        sh         0.637751    0.637721           -0.134673        -0.064709   
17      ch       701.840741  709.577536          299.868356       302.235610   
        db         0.697228    0.693872            0.893146         1.414191   

                                                                              \
method                 mica        kpod        mcnm        mghm       sc_knn   
dataset metric                                                                 
15      ch       978.721475  616.873124         NaN  668.709849   816.933131   
        db         0.771838    0.878015         NaN    0.917350     0.812214   
        sh         0.643339    0.582775         NaN    0.539803     0.625241   
17      ch      1286.222765  275.397839  902.651313  890.220372  1000.214556   
        db         0.505247    0.584524    0.602978    0.608042     0.569140   

                                                                                
method               sc_mi       km_knn        km_si cca_spectral   cca_kmeans  
dataset metric                                                                  
15      ch      824.009645   957.200347   978.799181   822.437456  1026.262388  
        db        0.808767     0.778323     0.771748     0.799292     0.757259  
        sh        0.627398     0.640223     0.643365     0.632756     0.651747  
17      ch      706.907172  1265.593009  1287.053596   738.656683  1300.208227  
        db        0.686116     0.509323     0.505357     0.671944     0.504404

## Ranks for internal metrics

In [17]:
# Calculate average rank for each method
LOWER_IS_BETTER = {"db"}

def rank_with_metric_direction(df):
    # this take second level index value at pos 0
    metric = df.index.get_level_values(1)[0] 
    return df.rank(
        axis="columns",
        ascending=metric in LOWER_IS_BETTER
    )

rank_mean = (
    combined_internal_means["value"][TESTED_METHODS]
    .groupby(level=1, group_keys=False)
    .apply(rank_with_metric_direction)
    .mean(axis=0)
)

rank_mean


method
mige_no_proj    4.666667
mige_proj       4.833333
mica            3.055556
kpod            5.722222
sc_knn          5.277778
sc_mi           5.611111
km_knn          4.277778
km_si           2.555556
dtype: float64

## Dump data

In [18]:
# dump 
combined_external_means.to_csv("../../test_output/analysis/combined_external_means.csv")
combined_external_stds.to_csv("../../test_output/analysis/combined_external_stds.csv")
combined_internal_means.to_csv("../../test_output/analysis/combined_internal_means.csv")
combined_internal_stds.to_csv("../../test_output/analysis/combined_internal_stds.csv")

# Statistical tests

In [19]:
def friedman_test(results, metrics):
    """
    """
    stat_res = {metric: None for metric in metrics}

    for metric in metrics:
        data = pd.DataFrame.from_dict(results[metric], orient='index')[TESTED_METHODS]
        stat, p = friedmanchisquare(*[data[col] for col in data.columns], nan_policy='omit')
        stat_res[metric] = p

    return stat_res

In [20]:
friedman_test(external_metrics_means, external_metrics)

{'ari': np.float64(0.012745030103727385),
 'ami': np.float64(0.00338005905362269),
 'vm': np.float64(0.0033068872560456063),
 'cs': np.float64(0.004296133781362652)}

In [21]:
friedman_test(internal_metrics_means, internal_metrics)

{'sh': np.float64(0.0011140131293388555),
 'ch': np.float64(0.00014810096267300378),
 'db': np.float64(9.495972508134177e-05)}

# Analysis stratified by missing ratios

In [22]:
missing_ratios = dict.fromkeys(set(configs))

for conf in tqdm(missing_ratios):
    dataset_id = conf[0]
    md_config = str(conf[1])+"_"+str(conf[2])+"_"+str(conf[3])
    seed = conf[4]
        
    """
    Load data for simulation
    """
    incomplete_data = read_pickle("../../test_data/missing_data/"+str(dataset_id)+"/"+md_config+"/data_pipeline_"+str(seed)+".pkl").amputer.incomplete_dataset
    missing_ratio = incomplete_data.isna().sum().sum()/(incomplete_data.shape[0] * incomplete_data.shape[1])
    missing_ratios[conf] = missing_ratio

  0%|          | 0/720 [00:00<?, ?it/s]

## Missing ratios frequencies

In [23]:
missing_stats = {k: [] for k in cfg['dataset_ids']}

for dataset_id in missing_stats.keys():
    key_subset = [k for k in missing_ratios.keys() if k[0] == dataset_id]
    for k in key_subset:
        missing_stats[dataset_id].append(missing_ratios[k])

In [24]:
# ax = sns.boxplot(pd.DataFrame(missing_stats),
#                  palette = sns.color_palette("muted"))


## Missing ratio binning

In [25]:
# general binning
# bin in 5% 

bins = np.arange(0,1,.05)

# dataset binned key order:    dataset: -> metric -> method -> bins ->  list of res "in" bin 
external_metrics_binned = {
    dataset_id:{
        metric:{
            method:{
                bin:[] for bin in bins
                } for method in TESTED_METHODS
            } for metric in external_metrics
        } for dataset_id in cfg['dataset_ids']
}

In [26]:
for cfg, v1 in simulation_results.items():

    dataset_id = cfg[0]
    bin = bins[np.digitize(missing_ratios[cfg], bins, right=True)]

    for method, v2 in v1['external_metrics'].items():
        if method in TESTED_METHODS:
            try:
                for metric, v3 in v2.items():
                    external_metrics_binned[dataset_id][metric][method][bin].append(v3)
            except:
                for metric in external_metrics:                 
                    external_metrics_binned[dataset_id][metric][method][bin].append(np.nan)
                


In [27]:
markers = cycle(['o', 's', '^', 'D', 'v', '>', '<', 'p', '*', 'h'])
linestyles = cycle(['-.',':'])
colors = cycle(sns.color_palette("tab10"))

style = {m:{} for m in methods}
for method in TESTED_METHODS:
    style[method]['marker'] = next(markers)
    style[method]['color'] = next(colors)
    if 'mige' in method:
        style[method]['linestyle'] = '-'
    elif 'cca' in method:
        style[method]['linestyle'] = '--'
    else:
        style[method]['linestyle'] = next(linestyles)

In [35]:
# load dataset names 

dataset_names = pd.read_csv("../../test_output/datasets_analysis.csv")
dataset_names.head(2)

,Unnamed: 0,Name,Id,Rows,Num_features,Num_categoricals,Num_classes,Missing_features,Missing_rows,Total_missings
0,0,Parkinsons,174,195,20,0,2,0,0,0
1,1,breast_cancer_wisconsin_diagnostic,17,569,30,0,2,0,0,0


In [59]:
# Example data
datasets = dataset_names['Id'].to_list()
metrics = external_metrics

for dataset, metric in product(datasets, metrics):

    dataset_name = dataset_names.loc[dataset_names['Id'] == dataset,'Name'].iloc[0]

    data = external_metrics_binned[dataset][metric]


    fig, ax = plt.subplots(figsize=(6.5, 4.5), dpi=300)

    for method, ratios in data.items():
        x = []
        y = []
        yerr = []

        for missing_ratio, values in sorted(ratios.items()):
            values = np.asarray(values)
            n = len(values)

            mean = np.mean(values)
            sem = stats.sem(values)
            ci95 = sem * stats.t.ppf(0.975, df=n - 1)

            x.append(missing_ratio)
            y.append(mean)
            yerr.append(ci95)

        ax.errorbar(
            x,
            y,
            yerr=yerr,
            marker=style[method]["marker"],
            linestyle=style[method]["linestyle"],
            linewidth=1.2,
            markersize=6,
            capsize=3,
            alpha = 0.6,
            label=method
        )

    # Axis labels (use units if applicable)
    ax.set_xlabel("Missing ratio", fontsize=12)
    ax.set_ylabel("Mean "+str.upper(metric), fontsize=12)

    # Ticks
    ax.tick_params(axis="both", labelsize=10)
    ax.set_title(str(dataset_name))

    # Grid: light and unobtrusive
    ax.grid(axis="y", linestyle="--", linewidth=0.6, alpha=0.4)
    ax.grid(axis="x", visible=False)

    # Legend
    ax.legend(
        title="Method",
        fontsize=10,
        title_fontsize=11,
        frameon=False,
        loc="center left",
        bbox_to_anchor=(1.02, 0.5)
    )
    fig.tight_layout()

    # Save for publication
    fig.savefig("../../test_output/figures/missing_ratio/"+dataset_name+str(dataset)+metric+".pdf")
    fig.savefig("../../test_output/figures/missing_ratio/"+dataset_name+str(dataset)+metric+".png")

    plt.close(fig);   # ← hides output


c:\Users\simone\anaconda3\envs\miclust\Lib\site-packages\numpy\_core\fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\simone\anaconda3\envs\miclust\Lib\site-packages\numpy\_core\_methods.py:144: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
C:\Users\simone\AppData\Local\Temp\ipykernel_28172\723064990.py:24: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  sem = stats.sem(values)
c:\Users\simone\anaconda3\envs\miclust\Lib\site-packages\numpy\_core\fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\simone\anaconda3\envs\miclust\Lib\site-packages\numpy\_core\_methods.py:144: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
C:\Users\simone\AppData\Local\Temp\ipykern

## MNAR 

In [63]:
# mnar percentage is the fourth elem in the cfg tuple
with open("../../test_data/simulation_config.yaml", "r") as f:
    cfg = yaml.safe_load(f)
mnar_bins = cfg['md_param_grid']['mnar_proportions']

external_metrics_mnar_results = {met:{d: {bin: {m:[] for m in methods} for bin in mnar_bins} for d in cfg['dataset_ids']} for met in external_metrics}
external_metrics_mnar_means = {met:{d: {bin: {m:np.nan for m in methods} for bin in mnar_bins} for d in cfg['dataset_ids']} for met in external_metrics}
external_metrics_mnar_stds = {met:{d: {bin: {m:np.nan for m in methods} for bin in mnar_bins} for d in cfg['dataset_ids']} for met in external_metrics}

for k1,v1 in simulation_results.items():
    # k1 is the cfg 
    mnar_bin = k1[3]
    dataset_id = k1[0]
    # collect results
    for metric in external_metrics:
        for method in methods:
            #res = v1['external_metrics'][method][metric]
            res = safe_get(v1['external_metrics'],method,metric)
            external_metrics_mnar_results[metric][dataset_id][mnar_bin][method].append(res)

# calculate means and standard deviations
for metric,v1 in external_metrics_mnar_results.items():
    for dataset_id, v2 in v1.items():
        for mnar_bin, v3 in v2.items():
            for method,v4 in v3.items():
                external_metrics_mnar_means[metric][dataset_id][mnar_bin][method] = np.mean(v4)
                external_metrics_mnar_stds[metric][dataset_id][mnar_bin][method] = np.std(v4)


In [66]:
df = (
    pd.DataFrame.from_dict(external_metrics_mnar_means['ari'], orient="index")
      .stack()
      .apply(pd.Series)
)

In [69]:
df[TESTED_METHODS].head(20)

mige_no_proj  mige_proj      mica      kpod    sc_knn     sc_mi  \
33  0.00      0.816059   0.829810  0.086995  0.013249  0.637389  0.673038   
    0.25      0.827113   0.827342  0.078738  0.011547  0.629847  0.665711   
    0.50      0.827503   0.828133  0.080230  0.011406  0.633642  0.665019   
45  0.00      0.110314   0.115006  0.040522  0.015243  0.094067  0.077286   
    0.25      0.118256   0.114744  0.054385  0.017667  0.085975  0.086169   
    0.50      0.117805   0.115477  0.055307  0.016535  0.094750  0.077394   
17  0.00      0.781466   0.778287  0.485174  0.085584  0.518525  0.576763   
    0.25      0.778322   0.777848  0.482959  0.085282  0.518675  0.574352   
    0.50      0.778796   0.777686  0.483603  0.085282  0.518675  0.573090   
15  0.00      0.855315   0.853671  0.831572  0.533426  0.677526  0.682286   
    0.25      0.853254   0.853120  0.829545  0.530342  0.672524  0.675666   
    0.50      0.854358   0.856693  0.827502  0.529968  0.671896  0.673173   
174 0.00      0.152489   0.150145  0.113747  0.007475 -0.002348 -0.035713   
    0.25      0.150814   0.150060  0.114215  0.006219 -0.007260 -0.035571   
    0.50      0.152269   0.149689  0.117240  0.006762 -0.007229 -0.035473   
544 0.00      0.146817   0.142689  0.240424  0.169947  0.193633  0.139939   
    0.25      0.147514   0.146531  0.243441  0.168833  0.198069  0.134030   
    0.50      0.147032   0.143695  0.243556  0.169230  0.195391  0.134303   

            km_knn     km_si  
33  0.00  0.079621  0.129228  
    0.25  0.078263  0.117694  
    0.50  0.083822  0.117766  
45  0.00  0.041161  0.054564  
    0.25  0.047432  0.059333  
    0.50  0.048103  0.058212  
17  0.00  0.484294  0.487224  
    0.25  0.485057  0.483856  
    0.50  0.485057  0.484636  
15  0.00  0.817153  0.833196  
    0.25  0.816477  0.830216  
    0.50  0.815281  0.829106  
174 0.00  0.122320  0.126239  
    0.25  0.126300  0.121517  
    0.50  0.119314  0.117378  
544 0.00  0.240681  0.238121  
    0.25  0.240459  0.238483  
    0.50  0.242000  0.239340